In [1]:
#notebook for s3 bucket, athena, dataset loading

In [2]:
#create s3 bucket

In [3]:
import boto3
import sagemaker
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
s3 = boto3.Session().client(service_name="s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-883727203134


In [5]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': '8Z9E4SZ9EP7CEHE9', 'HostId': 'kg8fZS7g6YP/++/LctzV/1za9JWrVdCvvr+PfpnNZuRTcq1GMGDBWYpwOudKL62Q+JAMwLDGZ0JW+/fA88XjZE4jhhygHb7a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'kg8fZS7g6YP/++/LctzV/1za9JWrVdCvvr+PfpnNZuRTcq1GMGDBWYpwOudKL62Q+JAMwLDGZ0JW+/fA88XjZE4jhhygHb7a', 'x-amz-request-id': '8Z9E4SZ9EP7CEHE9', 'date': 'Sun, 25 Jan 2026 20:23:19 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'x-amz-bucket-arn': 'arn:aws:s3:::sagemaker-us-east-1-883727203134', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [6]:
# copy over datasets to s3 

In [7]:
!aws s3 cp Natural-Questions-Base.csv s3://sagemaker-us-east-1-883727203134/Natural-Questions-Base.csv

upload: ./Natural-Questions-Base.csv to s3://sagemaker-us-east-1-883727203134/Natural-Questions-Base.csv


In [ ]:
!aws s3 cp Natural-Questions-Filtered.csv s3://sagemaker-us-east-1-883727203134/Natural-Questions-Filtered.csv

In [11]:
!aws s3 ls s3://sagemaker-us-east-1-883727203134/

                           PRE athena/
                           PRE sagemaker-featurestore-assignment3.1/
                           PRE table/
2026-01-20 05:18:32   20118244 dataset.csv
2026-01-25 14:46:45    1423529 housing.csv
2026-01-25 14:46:48    2057894 housing_gmaps_data_raw.csv
2026-01-25 15:44:35     573653 llm-df.csv


In [ ]:
# load more 

In [ ]:
# load more

In [ ]:
# load more

In [ ]:
# load more

In [ ]:
# create athena db for query 

In [12]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [13]:
from pyathena import connect

In [14]:
database_name = "dsoaws"

In [15]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [16]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [17]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS dsoaws


In [18]:
import pandas as pd
pd.read_sql(statement, conn)

/tmp/ipykernel_258/1322682284.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [19]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_258/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws
2,sagemaker_featurestore


In [20]:
database_name = "dsoaws"
llmtable_name_csv = "llmtable"

In [21]:
llmexpo = pd.read_csv('llm-df.csv')
llmexpo.head()

,T,Model,Average ⬆️,IFEval,IFEval Raw,BBH,BBH Raw,MATH Lvl 5,MATH Lvl 5 Raw,GPQA,...,Model sha,Flagged,MoE,Submission Date,Upload To Hub Date,Chat Template,Maintainer's Highlight,fullname,Generation,Base Model
0,💬,dfurman/CalmeRys-78B-Orpo-v0.1,50.78,81.63,0.82,61.92,0.73,37.92,0.38,20.02,...,7988deb48419c3f56bb24c139c23e5c476ec03f8,True,True,2024-09-24,2024-09-24,True,False,dfurman/CalmeRys-78B-Orpo-v0.1,1,dfurman/CalmeRys-78B-Orpo-v0.1 (Merge)
1,💬,MaziyarPanahi/calme-2.4-rys-78b,50.26,80.11,0.80,62.16,0.73,37.69,0.38,20.36,...,0a35e51ffa9efa644c11816a2d56434804177acb,True,True,2024-09-03,2024-08-07,True,False,MaziyarPanahi/calme-2.4-rys-78b,2,dnhkng/RYS-XLarge
2,🔶,rombodawg/Rombos-LLM-V2.5-Qwen-72b,45.39,71.55,0.72,61.27,0.72,47.58,0.48,19.80,...,5260f182e7859e13d515c4cb3926ac85ad057504,True,True,2024-09-30,2024-09-30,False,False,rombodawg/Rombos-LLM-V2.5-Qwen-72b,1,rombodawg/Rombos-LLM-V2.5-Qwen-72b (Merge)
3,🔶,dnhkng/RYS-XLarge,44.75,79.96,0.80,58.77,0.71,38.97,0.39,17.90,...,0f84dd9dde60f383e1e2821496befb4ce9a11ef6,True,True,2024-08-07,2024-07-24,False,False,dnhkng/RYS-XLarge,0,dnhkng/RYS-XLarge
4,💬,MaziyarPanahi/calme-2.1-rys-78b,44.14,81.36,0.81,59.47,0.71,36.40,0.36,19.24,...,e746f5ddc0c9b31a2382d985a4ec87fa910847c7,True,True,2024-08-08,2024-08-06,True,False,MaziyarPanahi/calme-2.1-rys-78b,1,dnhkng/RYS-XLarge
